# Adeptus Optimus Proto

# Calculus

In [11]:
import scipy.special
import numpy as np
import random
import re
import json

In [12]:
# Utils
def map_7_to_None(v):
    return None if v == 7 else v

class RequirementFailError(Exception):
    pass

def require(predicate, error_message):
    if not(predicate):
        raise RequirementFailError(error_message)

require(True, "bla")

thrown_message = ""
try:
    require(False, "bla")
except RequirementFailError as e:
    thrown_message = str(e)
assert(thrown_message == "bla")
    
    
    
def roll_D6():
    return random.randint(1, 6)
assert(len({roll_D6() for _ in range(1000)}) == 6)

def roll_D3():
    return (random.randint(1, 6) + 1) // 2
assert(len({roll_D3() for _ in range(1000)}) == 3)

class DiceExpr:
    def __init__(self, n, dices_type=None):
        self.n = n
        self.dices_type = dices_type
        
        if self.dices_type is None:
            self.avg = n
        else:
            self.avg = n * (self.dices_type + 1)/2
    
    def roll(self):
        if self.dices_type is None:
            return self.n
        else:
            if self.dices_type == 3:
                return sum([roll_D3() for _ in range(self.n)])
            elif self.dices_type == 6:
                return sum([roll_D6() for _ in range(self.n)])
            else:
                raise AttributeError(f"Unsupported dices_type D{self.dices_type}")
    
        
    def __str__(self):
        if self.dices_type is None:
            return str(self.n)
        else:
            return f"{self.n if self.n > 1 else ''}D{self.dices_type}"

        
assert(str(DiceExpr(5, 3)) == "5D3")
assert(str(DiceExpr(1, 6)) == "D6")
assert(str(DiceExpr(10, None)) == "10")

dice_5D3 = DiceExpr(5, 3)
assert(len({dice_5D3.roll() for _ in range(10000)}) == 5*3 -5 + 1)
dice_4D6 = DiceExpr(4, 6)
assert(len({dice_4D6.roll() for _ in range(10000)}) == 4*6 -4 + 1)          
    
def parse_dice_expr(d, complexity_threshold=36):
    assert(type(d) is str)
    groups = re.fullmatch(r"([1-9][0-9]*)?D([36])?|([0-9]+)", d)
    res = None
    try:
        if groups.group(1) is None:
            n_dices = 1
        else:
            n_dices = int(groups.group(1))
        dices_type = int(groups.group(2))
        if(n_dices*(1 if dices_type is None else dices_type) > complexity_threshold):
            res = None
        else:
            res = DiceExpr(n_dices, dices_type)
    except TypeError:
        try:
            flat = int(groups.group(3))
            res = DiceExpr(flat)
        except TypeError:
            res = None
    finally:
        return res
    
assert(parse_dice_expr("4D3").avg == 8)
assert(parse_dice_expr("5").avg == 5)
assert(parse_dice_expr("D7") is None)
assert(parse_dice_expr("0D6") is None)
assert(parse_dice_expr("0").avg == 0)
assert(parse_dice_expr("7D6") is None)
assert(parse_dice_expr("D3").avg == 2)
assert(parse_dice_expr("3D3").avg == 6)
assert(parse_dice_expr("D6").avg == 3.5)

def parse_roll(roll):
    res = re.fullmatch(r"([23456])\+", roll)
    if res is None:
        return None
    else:
        return int(res.group(1))

assert(parse_roll("1+") is None)
assert(parse_roll("1+") is None)
assert(parse_roll("2+") == 2)
assert(parse_roll("3+") == 3)
assert(parse_roll("6+") == 6)
assert(parse_roll("7+") is None)
assert(parse_roll("3") is None)

def float_eq(a, b):
    return np.isclose(a, b, rtol=1e-05, atol=1e-08, equal_nan=False)

def prob_by_roll_result(dice_expr):
    if dice_expr.dices_type is None:
        return {dice_expr.n: 1}
    else:
        roll_results_counts = {}
        def f(n, current_sum):
            if n == 0:
                roll_results_counts[current_sum] = roll_results_counts.get(current_sum, 0) + 1
            else:
                for i in range(1, dice_expr.dices_type + 1):
                    f(n - 1, current_sum + i)
        f(dice_expr.n, 0)
        n_cases = sum(roll_results_counts.values())
        for key in roll_results_counts.keys():
            roll_results_counts[key] /= n_cases
        return roll_results_counts
    
assert(prob_by_roll_result(parse_dice_expr("D3")) == {1: 1/3, 2: 1/3, 3: 1/3})
assert(prob_by_roll_result(parse_dice_expr("7")) == {7: 1})
assert(float_eq(1, sum(prob_by_roll_result(parse_dice_expr("2D6")).values())))
assert(prob_by_roll_result(parse_dice_expr("2D6")) == {2: 1/36, 3: 2/36, 4: 3/36, 5: 4/36, 6: 5/36, 7: 6/36, 8: 5/36, 9: 4/36, 10: 3/36, 11: 2/36, 12: 1/36})

In [13]:
# Core Classes
class Bonuses:
    def __init__(self, to_hit, to_wound, props = None):
        assert(to_hit in {-1, 0, 1})
        assert(to_wound in {-1, 0, 1})

        self.to_hit = to_hit
        self.to_wound = to_wound
        self.props = props if props is not None else {}
        
    @classmethod
    def empty(cls):
        return Bonuses(0, 0)
    
Bonuses.empty().to_hit = 0
        
class Weapon:
    def __init__(self, hit, a, s, ap, d, bonuses, points=1):
        self.hit      = parse_dice_expr(hit)
        assert(self.hit is not None)
        self.a        = parse_dice_expr(a)
        assert(self.a is not None)
        self.s        = parse_dice_expr(s)
        assert(self.s is not None)
        self.ap       = parse_dice_expr(ap)
        assert(self.ap is not None)
        self.d        = parse_dice_expr(d)
        assert(self.d is not None)

        self.bonuses  = bonuses   
        assert(type(points) is int and points > 0)
        self.points   = points
        
        
Weapon(hit="5", a="2", s = "4D3", ap = "1", d="D3", bonuses=Bonuses.empty())

        
class Target:
    def __init__(self, t, sv, invu=None, fnp=None, w=1):

        
        assert(invu is None or (type(invu) is int and invu > 0 and invu <= 6))
        self.invu   = invu
        
        assert(fnp is None or (type(fnp) is int and fnp > 0 and fnp <= 6))
        self.fnp   = fnp
        
        assert(type(t) is int and t > 0)
        self.t    = t
        
        assert(type(sv) is int and sv > 0 and sv <= 6)
        self.sv   = sv
        
        assert(type(w) is int and w > 0)
        self.w   = w


Target(8, 3, 5, 6)
Target(8, 3)

In [14]:
# Engine v1

def compute_successes_ratio(modified_necessary_roll, auto_success_on_6=True):
    necessary_roll = modified_necessary_roll
    if modified_necessary_roll <= 1:
        necessary_roll = 2  # roll of 1 always fails
    if modified_necessary_roll >= 7:
        if auto_success_on_6:
            necessary_roll = 6  # roll of 6 always succeeds
        else:
            return 0
    return (7 - necessary_roll)/6

def compute_n_hits_ratio(weapon):
    return compute_successes_ratio(weapon.hit.avg - weapon.bonuses.to_hit)


assert(compute_n_hits_ratio(Weapon(hit="2", a="12", s = "4", ap = "1", d="2D3", bonuses=Bonuses.empty())) == 5/6)
assert(compute_n_hits_ratio(Weapon(hit="5", a="12", s = "4", ap = "1", d="2D3", bonuses=Bonuses.empty())) == 1/3)
assert(compute_n_hits_ratio(Weapon(hit="3", a="1", s = "4", ap = "1",  d="2D3", bonuses=Bonuses.empty())) == 2/3)
assert(compute_n_hits_ratio(Weapon(hit="3", a="1", s = "4", ap = "1",  d="2D3", bonuses=Bonuses(-1, 0))) == 1/2)
assert(compute_n_hits_ratio(Weapon(hit="2", a="1", s = "4", ap = "1",  d="2D3", bonuses=Bonuses(+1, 0))) == 5/6)  # roll of 1 always fails
assert(compute_n_hits_ratio(Weapon(hit="6", a="1", s = "4", ap = "1",  d="2D3", bonuses=Bonuses(-1, 0))) == 1/6)  # roll of 6 always succeeds

def compute_necessary_wound_roll(f, e):
    if f >= 2*e:
        return 2
    elif f > e:
        return 3
    elif f == e:
        return 4
    elif f <= e/2:
        return 6
    else:
        assert(f < e)
        return 5

def compute_n_wounds_ratios(weapon, target):
    return [prob*compute_successes_ratio(compute_necessary_wound_roll(f, target.t) - weapon.bonuses.to_wound)
     for f, prob in prob_by_roll_result(weapon.s).items()]

def compute_n_wounds_ratio(weapon, target):
    return sum(compute_n_wounds_ratios(weapon, target))

assert(compute_n_wounds_ratio(
    Weapon(hit="2", a="1", s = "4", ap = "1", d="2D3", bonuses=Bonuses(+1, 0)),
    Target(t=4, sv=3)
) == 1/2)

assert(compute_n_wounds_ratio(
    Weapon(hit="2", a="12", s = "4", ap = "1", d="2D3", bonuses=Bonuses(+1, 0)),
    Target(t=8, sv=3)
) == 1/6)

assert(compute_n_wounds_ratio(
    Weapon(hit="2", a="1", s = "4", ap = "1", d="2D3", bonuses=Bonuses(+1, -1)),
    Target(t=8, sv=3)
) == 1/6)  # roll of 6 always succeeds

assert(compute_n_wounds_ratio(
    Weapon(hit="2", a="1", s = "4", ap = "1", d="2D3", bonuses=Bonuses(+1, 0)),
    Target(t=2, sv=3)
) == 5/6)

assert(compute_n_wounds_ratio(
    Weapon(hit="2", a="1", s = "4", ap = "1", d="2D3", bonuses=Bonuses(+1, +1)),
    Target(t=2, sv=3)
) == 5/6)  # roll of 1 always fails

assert(compute_n_wounds_ratio(
    Weapon(hit="2", a="1", s = "2D3", ap = "1", d="2D3", bonuses=Bonuses(+1, 0)),
    Target(t=12, sv=3)
) == 1/6)

assert(float_eq(compute_n_wounds_ratio(
    Weapon(hit="2", a="1", s = "2D3", ap = "1", d="2D3", bonuses=Bonuses(+1, 0)),
    Target(t=1, sv=3)
), 5/6))

assert(compute_n_wounds_ratio(
    Weapon(hit="2", a="1", s = "2D3", ap = "1", d="2D3", bonuses=Bonuses(+1, 0)),
    Target(t=4, sv=3)
)< 1/2)



def n_figs_slained(weapon, target):
    hit_ratio = compute_n_hits_ratio(weapon)
    wound_ratio = compute_n_wounds_ratio(weapon, target)
    return weapon.a.avg * hit_ratio * wound_ratio

def total_n_figs_slained(weapons_list, target):
    print("tatata")

In [15]:
n_figs_slained(
    Weapon(hit="5", a="12", s = "4", ap = "1", d="2D3", bonuses=Bonuses(+1, 0)),
    Target(t=8, sv=3)
)

1.0

In [16]:
# Engine v2
def dispatch_density_key(previous_density_key, next_density_prob):
    assert(type(previous_density_key) is int)
    assert(previous_density_key >= 0)
    assert(0 < next_density_prob and next_density_prob <= 1)
    n = previous_density_key
    p= next_density_prob
    return {k: scipy.special.comb(n, k)*p**k*(1-p)**(n-k) for k in range(0, n + 1)}

assert(dispatch_density_key(3, 0.5) == {0: 0.125, 1: 0.375, 2: 0.375, 3: 0.125})
# new version:
# 3 A, c 4, f 4 endu 4
# [0, 0, 0, 1] attacks density
def get_attack_density(weapon):
    assert(isinstance(weapon, Weapon))
    return {a: prob for a, prob in prob_by_roll_result(weapon.a).items()}
# [1/8, 3/8, 3/8 ,1/8] hit density
def get_hits_density(weapon, attack_density):
    assert(isinstance(weapon, Weapon))
    assert(isinstance(attack_density, dict))
    hits_density = {}
    for a, prob_a in attack_density.items(): 
        # {1: 0.3333333333333333, 2: 0.3333333333333333, 3: 0.3333333333333333}
        for hit_roll, prob_hit_roll in prob_by_roll_result(weapon.hit).items():
            # {5: 1}
            hits_ratio = compute_successes_ratio(hit_roll - weapon.bonuses.to_hit)
            # 0.5
            for hits, prob_hits in dispatch_density_key(a, hits_ratio).items():
                hits_density[hits] = hits_density.get(hits, 0) + prob_hits*prob_hit_roll*prob_a
    return hits_density

# [......]  woud density
def get_wounds_density(weapon, target, hits_density):
    assert(isinstance(weapon, Weapon))
    assert(isinstance(target, Target))
    assert(isinstance(hits_density, dict))
    wounds_density = {}
    for hits, prob_hits in hits_density.items(): 
        for s_roll, prob_s_roll in prob_by_roll_result(weapon.s).items():
            wounds_ratio = compute_successes_ratio(compute_necessary_wound_roll(s_roll, target.t) - weapon.bonuses.to_wound)
            for wounds, prob_wounds in dispatch_density_key(hits, wounds_ratio).items():
                wounds_density[wounds] = wounds_density.get(wounds, 0) + prob_wounds*prob_s_roll*prob_hits
    return wounds_density

# [......] unsaved wounds density
def get_unsaved_wounds_density(weapon, target, wounds_density):
    assert(isinstance(weapon, Weapon))
    assert(isinstance(target, Target))
    assert(isinstance(wounds_density, dict))
    unsaved_wounds_density = {}
    for wounds, prob_wounds in wounds_density.items(): 
        for ap_roll, prob_ap_roll in prob_by_roll_result(weapon.ap).items():
            save_roll = target.sv + ap_roll
            if target.invu is not None:
                save_roll = min(save_roll, target.invu)
            unsaved_wounds_ratio = 1 - compute_successes_ratio(save_roll, auto_success_on_6=False)
            for unsaved_wounds, prob_unsaved_wounds in dispatch_density_key(wounds, unsaved_wounds_ratio).items():
                unsaved_wounds_density[unsaved_wounds] = unsaved_wounds_density.get(unsaved_wounds, 0) + prob_unsaved_wounds*prob_ap_roll*prob_wounds
    return unsaved_wounds_density

# last step numeric averaging: damage roll + fnp
def get_avg_figs_fraction_slained(weapon, target, unsaved_wounds_density, N):
    """
    El famoso montecarlo approach
    :param N: number of simulations for each unsaved wounds entry
    """
    assert(isinstance(weapon, Weapon))
    assert(isinstance(target, Target))
    assert(isinstance(unsaved_wounds_density, dict))
    figs_fraction_slained_total_weighted = 0
    adapted_N = (N // target.w) + 1
    for unsaved_wounds, prob_unsaved_wounds in unsaved_wounds_density.items():
        figs_fraction_slained_total_sum = 0
        for _ in range(adapted_N):
            figs_fraction_slained_total = 0
            for start_health in range(1, target.w + 1):
                n_figs_slained = 0
                remaining_health = start_health
                for wound in range(unsaved_wounds):
                    damages = weapon.d.roll()
                    if target.fnp is not None:
                        for damage in range(damages):
                            if roll_D6() >= target.fnp:
                                damages -= 1  # fnp success
                    remaining_health -= damages
                    if remaining_health <= 0:
                        n_figs_slained += 1
                        remaining_health = target.w
                # e.g. remaining = 1,slained 2, w=3, start = 2: frac = 2 - (1 - 2/3) + (1 - 1/3) 
                first_slain_fraction = start_health/target.w
                remaining_fraction = remaining_health/target.w
                figs_fraction_slained_total += \
                    n_figs_slained - (1 - first_slain_fraction) + (1 - remaining_fraction)

            figs_fraction_slained_total_sum += figs_fraction_slained_total

        figs_fraction_slained_total_weighted += \
            prob_unsaved_wounds * figs_fraction_slained_total_sum/(adapted_N*target.w) # target_wounds is here because of start_health loop

    return figs_fraction_slained_total_weighted

def score_weapon_on_target(w, t, N):
    """
    avg_figs_fraction_slained by point
    """
    a_d = get_attack_density(w)
    assert(float_eq(sum(a_d.values()), 1))
    h_d = get_hits_density(w, a_d)
    assert(float_eq(sum(h_d.values()), 1))
    w_d = get_wounds_density(w, t, h_d)
    assert(float_eq(sum(w_d.values()), 1))
    uw_d = get_unsaved_wounds_density(w, t, w_d)
    assert(float_eq(sum(uw_d.values()), 1))
    return get_avg_figs_fraction_slained(w, t, uw_d, N)/w.points

# Sv=1 : ignore PA -1
wea = Weapon(hit="4", a="4", s = "4", ap = "1", d="3", bonuses=Bonuses(0, 0), points=120)
wea2 = Weapon(hit="4", a="4", s = "4", ap = "0", d="3", bonuses=Bonuses(0, 0), points=120)
tar = Target(t=4, sv=1, invu=5, fnp=6, w=16)
assert(abs(score_weapon_on_target(wea, tar, 1000) / score_weapon_on_target(wea2, tar, 100) - 1) <= 0.25)

def scores_to_comparison_score(score_a, score_b):
    if score_a > score_b:
        return + (1 - score_b / score_a)
    else:
        return - (1 - score_a / score_b)

assert(scores_to_comparison_score(10000, 1) == 0.9999) 
assert(scores_to_comparison_score(1, 10000) == -0.9999) 
assert(scores_to_comparison_score(1, 1) == 0) 

def y_dims_to_str(l):
    return f"""{l[0]}/{l[1]}/{"-" if l[2] is None else f"{l[2]}+"}"""

def x_dims_to_str(l):
    return  f"""{"-" if l[0] is None else f"{l[0]}+"}/{"-" if l[1] is None else f"{l[1]}+"}"""
def compute_matrix(weapon_a, weapon_b, N):    
    ws_ts_fnps = []
    for w, ts in zip(
        [1,2,3,4,6,8,10,12,16], 
        [
            [2,3,4],
            [2,3,4,5],
            [2,3,4,5,6],
            [2,3,4,5,6],
            [3,4,5,6,8],
            [4,5,6,8],
            [5,6,8],
            [6,8],
            [6,8]
        ]
    ) :
        fnps = [7] if w > 6 else [7, 6, 5]
        for fnp in fnps:
            for t in ts:
                ws_ts_fnps.append((t, w, fnp))
                
    ws_ts_fnps.sort(key=lambda e: e[2] * 10000 - e[0]*100 - e[1])
    ws_ts_fnps = list(map(lambda l: list(map(map_7_to_None, l)), ws_ts_fnps))
    
    print(list(map(y_dims_to_str, ws_ts_fnps)), )
    
    svs = []
    for invu in [2, 3, 4, 5, 6, 7]:
        for sv in range(1, min(invu + 1, 6 + 1)):
            svs.append((sv, invu))
    svs.sort(key=lambda e: -e[0]*10 + -e[1])
    svs = list(map(lambda l: list(map(map_7_to_None, l)), svs))
    
    print(list(map(x_dims_to_str, svs)))
    
    
    return  \
        [
            [
                scores_to_comparison_score(
                    score_weapon_on_target(
                        weapon_a, 
                        Target(t, sv, invu=invu, fnp=fnp, w=w),
                        N), 
                    score_weapon_on_target(
                    weapon_b, 
                    Target(t, sv, invu=invu, fnp=fnp, w=w),
                    N)
                )
                for sv, invu in svs
            ]
            for w, t, fnp in ws_ts_fnps
        ]

In [17]:
# SAG vs Battlewagon under KFF and fnp 6+
sag = Weapon(hit="5", a="2D6", s = "2D6", ap = "5", d="D6", bonuses=Bonuses(0, 0), points=120)
bat = Target(t=8, sv=4, invu=5, fnp=6, w=16)

# SAG vs Battlewagon under KFF and fnp 6+
bolt = Weapon(hit="3", a="2", s = "4", ap = "1", d="1", bonuses=Bonuses(0, 0), points=20)

In [336]:
compute_matrix(sag, bolt, 1000)

['8/6/5+', '6/6/5+', '6/4/5+', '6/3/5+', '5/6/5+', '5/4/5+', '5/3/5+', '5/2/5+', '4/6/5+', '4/4/5+', '4/3/5+', '4/2/5+', '4/1/5+', '3/6/5+', '3/4/5+', '3/3/5+', '3/2/5+', '3/1/5+', '2/4/5+', '2/3/5+', '2/2/5+', '2/1/5+', '8/6/6+', '6/6/6+', '6/4/6+', '6/3/6+', '5/6/6+', '5/4/6+', '5/3/6+', '5/2/6+', '4/6/6+', '4/4/6+', '4/3/6+', '4/2/6+', '4/1/6+', '3/6/6+', '3/4/6+', '3/3/6+', '3/2/6+', '3/1/6+', '2/4/6+', '2/3/6+', '2/2/6+', '2/1/6+', '8/16/-', '8/12/-', '8/10/-', '8/8/-', '8/6/-', '6/16/-', '6/12/-', '6/10/-', '6/8/-', '6/6/-', '6/4/-', '6/3/-', '5/10/-', '5/8/-', '5/6/-', '5/4/-', '5/3/-', '5/2/-', '4/8/-', '4/6/-', '4/4/-', '4/3/-', '4/2/-', '4/1/-', '3/6/-', '3/4/-', '3/3/-', '3/2/-', '3/1/-', '2/4/-', '2/3/-', '2/2/-', '2/1/-']
['6+/-', '6+/6+', '5+/-', '5+/6+', '5+/5+', '4+/-', '4+/6+', '4+/5+', '4+/4+', '3+/-', '3+/6+', '3+/5+', '3+/4+', '3+/3+', '2+/-', '2+/6+', '2+/5+', '2+/4+', '2+/3+', '2+/2+', '1+/-', '1+/6+', '1+/5+', '1+/4+', '1+/3+', '1+/2+']


In [341]:
import numpy as np
%time print(compute_matrix(sag, bolt, 1000))

['8/6/5+', '6/6/5+', '6/4/5+', '6/3/5+', '5/6/5+', '5/4/5+', '5/3/5+', '5/2/5+', '4/6/5+', '4/4/5+', '4/3/5+', '4/2/5+', '4/1/5+', '3/6/5+', '3/4/5+', '3/3/5+', '3/2/5+', '3/1/5+', '2/4/5+', '2/3/5+', '2/2/5+', '2/1/5+', '8/6/6+', '6/6/6+', '6/4/6+', '6/3/6+', '5/6/6+', '5/4/6+', '5/3/6+', '5/2/6+', '4/6/6+', '4/4/6+', '4/3/6+', '4/2/6+', '4/1/6+', '3/6/6+', '3/4/6+', '3/3/6+', '3/2/6+', '3/1/6+', '2/4/6+', '2/3/6+', '2/2/6+', '2/1/6+', '8/16/-', '8/12/-', '8/10/-', '8/8/-', '8/6/-', '6/16/-', '6/12/-', '6/10/-', '6/8/-', '6/6/-', '6/4/-', '6/3/-', '5/10/-', '5/8/-', '5/6/-', '5/4/-', '5/3/-', '5/2/-', '4/8/-', '4/6/-', '4/4/-', '4/3/-', '4/2/-', '4/1/-', '3/6/-', '3/4/-', '3/3/-', '3/2/-', '3/1/-', '2/4/-', '2/3/-', '2/2/-', '2/1/-']
['6+/-', '6+/6+', '5+/-', '5+/6+', '5+/5+', '4+/-', '4+/6+', '4+/5+', '4+/4+', '3+/-', '3+/6+', '3+/5+', '3+/4+', '3+/3+', '2+/-', '2+/6+', '2+/5+', '2+/4+', '2+/3+', '2+/2+', '1+/-', '1+/6+', '1+/5+', '1+/4+', '1+/3+', '1+/2+']
[[0.2900064411566179, 0.32

In [242]:
w=len([1,2,3,4,6,8,10,12,16])
t=len([2,3,4,5,6,8])
svs=len(range(1, 19 + 1))
fnp=len(range(1, 2 + 1))
w*t*svs*fnp  # 3200

2052

In [35]:
w_t_fnp = sum([
    1 * len([2,3,4])*2,      #w=1 # w * t possible *2 if fnp variant
    1 * len([2,3,4,5])*2,    #w=2
    1 * len([2,3,4,5,6])*2,  #w=3
    1 * len([2,3,4,5,6])*2,  #w=4
    1 * len([3,4,5,6,8])*2,#w=6
    1 * len([4,5,6,8]),#w=8
    1 * len([5,6,8]),#w=10
    1 * len([6,8]),#w=16
    1 * len([6,8]) #w=28
])
w_t_fnp

55

In [36]:
svs*w_t_fnp

1045

In [32]:
%timeit score_weapon_on_target(bolt, bat, 200)

1.11 ms ± 5.79 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [33]:
%timeit score_weapon_on_target(sag, bat, 200)

75.9 ms ± 538 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [38]:
1*1045/1000/60

0.017416666666666667

In [ ]:
# put FNP only on endu < 7

In [275]:
score_weapon_on_target(sag, bat, 1000), score_weapon_on_target(bolt, bat, 1000)

(0.014493157072157503, 0.021757435705378997)

In [198]:
w = sag
t = bat

In [190]:
a_d = get_attack_density(w)
a_d

{2: 0.027777777777777776,
 3: 0.05555555555555555,
 4: 0.08333333333333333,
 5: 0.1111111111111111,
 6: 0.1388888888888889,
 7: 0.16666666666666666,
 8: 0.1388888888888889,
 9: 0.1111111111111111,
 10: 0.08333333333333333,
 11: 0.05555555555555555,
 12: 0.027777777777777776}

In [191]:
h_d = get_hits_density(w, a_d)
h_d, sum(h_d.values())

({0: 0.09245826180349491,
  1: 0.2239853530307222,
  2: 0.26620813348361666,
  3: 0.21083128491947162,
  4: 0.12367835752228379,
  5: 0.05596105682474631,
  6: 0.019891044662844356,
  7: 0.005563176345069349,
  8: 0.0012047433299274988,
  9: 0.00019485804737601263,
  10: 2.210969797211731e-05,
  11: 1.5680636859657664e-06,
  12: 5.226878953219221e-08},
 1.0000000000000002)

In [192]:
w_d = get_wounds_density(w, t, h_d)
w_d, sum(w_d.values())

({0: 0.4029500867053239,
  1: 0.3367914778591636,
  2: 0.1689350056321175,
  3: 0.06478956365764589,
  4: 0.020154421512724424,
  5: 0.00512477251439623,
  6: 0.0010565739623414905,
  7: 0.00017367555510426272,
  8: 2.216603219186902e-05,
  9: 2.1100352269821476e-06,
  10: 1.405806713069297e-07,
  11: 5.83937585822815e-09,
  12: 1.1371713614420627e-10},
 1.0000000000000004)

In [193]:
uw_d = get_unsaved_wounds_density(w, t, w_d)
uw_d, sum(uw_d.values())

({0: 0.5366555241737849,
  1: 0.3162275735568333,
  2: 0.11078687649773596,
  3: 0.029102035242819145,
  4: 0.006064610751263366,
  5: 0.0010130132279732571,
  6: 0.00013499857838335794,
  7: 1.415632468850579e-05,
  8: 1.1407205971613996e-06,
  9: 6.802863136706233e-08,
  10: 2.8236457380877304e-09,
  11: 7.27678850725364e-11,
  12: 8.764573859500293e-13},
 1.0000000000000007)

# New last step

In [79]:
# last step numeric averaging: damage roll + fnp
def get_avg_figs_fraction_slained_per_unsaved_wound(weapon, target, N):
    """
    El famoso montecarlo approach
    :param N: number of consecutive wounds resolved: N=1000 leads to a result precise at +- 1.5%
    """
    assert (isinstance(weapon, Weapon))
    assert (isinstance(target, Target))
    n_figs_slained = 0
    remaining_health = target.w
    for _ in range(N):
        damages = weapon.d.roll()
        if target.fnp is not None:
            for damage in range(damages):
                if roll_D6() >= target.fnp:
                    damages -= 1  # fnp success
        remaining_health -= damages
        if remaining_health <= 0:
            n_figs_slained += 1
            remaining_health = target.w
    # e.g. remaining = 1,slained 2, w=3, frac = 2 + (1 - 1/3) 
    remaining_fraction = remaining_health / target.w
    return (n_figs_slained + (1 - remaining_fraction))/N

run = [get_avg_figs_fraction_slained_per_unsaved_wound(sag, bat, 1000) for _ in range(100)]
sum(run)/len(run), np.std(np.array(run)), np.std(run)/(sum(run)/len(run))

(0.16783812500000003, 0.0028725681970799216, 0.017115111343623035)

In [80]:
%timeit get_presimu_avg_figs_fraction_slained_by_unsaved_wound(bolt, bat, 1000)

1.11 ms ± 4.43 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [75]:
get_presimu_avg_figs_fraction_slained_by_unsaved_wound(bolt, bat, 2000)

0.052375